In [145]:
import sys
#import cPickle as pickle
import pickle
from datetime import datetime

In [146]:
def convert_to_icd9(dxStr):
	if dxStr.startswith('E'):
		if len(dxStr) > 4: return dxStr[:4] + '.' + dxStr[4:]
		else: return dxStr
	else:
		if len(dxStr) > 3: return dxStr[:3] + '.' + dxStr[3:]
		else: return dxStr

In [95]:
def convert_to_3digit_icd9(dxStr):
    if str(dxStr)=="nan": return dxStr
    if dxStr.startswith('E'):
        if len(dxStr) > 4: return dxStr[:4]
        else: return dxStr
    else:
        if len(dxStr) > 3: return dxStr[:3]
        else: return dxStr

In [96]:
import pandas as pd

In [155]:
df = pd.read_csv("../hdd0/data/R301_CD.csv")

/home/katie/anaconda3/envs/nhri/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (1,4,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [156]:
nonzeroids = df["ACODE_ICD9_1"].dropna().index

In [157]:
df["ACODE_ICD9_1"] = df["ACODE_ICD9_1"].astype(str)
df["ACODE_ICD9_2"] = df["ACODE_ICD9_2"].astype(str)
df["ACODE_ICD9_3"] = df["ACODE_ICD9_3"].astype(str)

In [158]:
len(nonzeroids)

7695541

In [101]:
import tqdm

In [159]:
print("filtering those too short records")
validpatients=[]
patients=df["ID"].unique()
for ids in tqdm.tqdm_notebook(patients[:100]):
    if sum(df["ID"]==ids)>10:
        validpatients.append(ids)

filtering those too short records


/tmp/ipykernel_479832/2585671106.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for ids in tqdm.tqdm_notebook(patients[:100]):


  0%|          | 0/100 [00:00<?, ?it/s]

In [147]:
len(patients)

39807

In [148]:
len(validpatients)

100

In [160]:
df = df[df["ID"].isin(validpatients)]

In [132]:
len(df["ID"].unique())

100

In [106]:
#ID->visit ID
#VISITID->FUNC_DATE 

In [192]:
print('Building pid-admission mapping, admission-date mapping')
pidAdmMap = {}
admDateMap = {}
for i in range(len(df)):
    tokens = df.iloc[i]
    pid = tokens[8]
    #admId = tokens[2] #this is based on user id
    admId = int(i)
    #admTime = datetime.strptime(str(tokens[6]),"%Y-%m-%d")
    admTime = tokens[6][:7]
    admDateMap[admId] = admTime
    if pid in pidAdmMap:
        pidAdmMap[pid].append(admId)
    else:
        pidAdmMap[pid]=[admId]

Building pid-admission mapping, admission-date mapping


In [270]:
len(admDateMap)

41234

In [271]:
print('Building admission-dxList mapping')
admDxMap = {}
#admDxMap_3digit = {}
for i in range(len(df)):
    tokens = df.iloc[i]
    #admId = tokens[2]
    admId = int(i)
    dxStr = convert_to_3digit_icd9(tokens[9])
    dxStr2 = convert_to_3digit_icd9(tokens[10])
    dxStr3 = convert_to_3digit_icd9(tokens[11])
    #dxStr = 'D_' + convert_to_icd9(tokens[4][1:-1]) ############## Uncomment this line and comment the line below, if you want to use the entire ICD9 digits.

    if str(dxStr2)!="nan" and str(dxStr3)!="nan": 
        admDxMap[admId] = [dxStr,dxStr2,dxStr3]
    elif str(dxStr3)!="nan":
        admDxMap[admId] = [dxStr,dxStr2]
    else:
        admDxMap[admId] = [dxStr]

#tokens[4]後面原先有一個[1:-1], 看不出作用所以被我移除了


Building admission-dxList mapping


In [184]:
len(admDxMap)

41234

In [272]:
print('Building pid-sortedVisits mapping')
pidSeqMap = {}
pidSeqMap_3digit = {}
for pid, admIdList in pidAdmMap.items():
    if len(admIdList) < 2: 
        continue

    sortedList = sorted([(admDateMap[admId], admDxMap[admId]) for admId in admIdList])
    pidSeqMap[pid] = sortedList

    #sortedList_3digit = sorted([(admDateMap[admId], admDxMap_3digit[admId]) for admId in admIdList])
    #pidSeqMap_3digit[pid] = sortedList_3digit
#visit to disease

Building pid-sortedVisits mapping


In [273]:
print("Buliding new pid-seq-map")
new_pidSeqMap={}
for pid in pidSeqMap.keys():
    new_pidSeqMap[pid]=pd.DataFrame(pidSeqMap[pid]).groupby(0).sum().reset_index().values

Buliding new pid-seq-map


In [174]:
print('Building pid-sortedVisits mapping')
pidSeqMap = {}
for pid, admIdList in pidAdmMap.items():
    sortedList = sorted([(admDateMap[admId], admDxMap[admId]) for admId in admIdList])
    pidSeqMap[pid] = sortedList

#visit to disease

Building pid-sortedVisits mapping


In [274]:
print('Building pids, dates, strSeqs')
pids = []
dates = []
seqs = []
for pid, visits in new_pidSeqMap.items():
    pids.append(pid)
    seq = []
    date = []
    for visit in visits:
        date.append(visit[0])
        seq.append(visit[1])
    dates.append(date)
    seqs.append(seq)

Building pids, dates, strSeqs


In [275]:
print('Converting strSeqs to intSeqs, and making types')
types = {}
newSeqs = []
for patient in seqs:
    newPatient = []
    for visit in patient:
        newVisit = []
        for code in visit:
            if code in types:
                newVisit.append(types[code])
            else:
                types[code] = len(types)
                newVisit.append(types[code])
        newPatient.append(newVisit)
    newSeqs.append(newPatient)

Converting strSeqs to intSeqs, and making types


In [276]:
print('Re-formatting to Med2Vec dataset')
seqs = []
for patient in newSeqs:
    seqs.extend(patient)
    seqs.append([-1])
seqs = seqs[:-1]

Re-formatting to Med2Vec dataset


In [282]:
outFile="./mimic_demo/ourR301_100_new"

In [283]:
pickle.dump(seqs, open(outFile+'.seqs', 'wb'), -1)
pickle.dump(types, open(outFile+'.types', 'wb'), -1)
#pickle.dump(seqs_3digit, open(outFile+'.3digitICD9.seqs', 'wb'), -1)
#pickle.dump(types_3digit, open(outFile+'.3digitICD9.types', 'wb'), -1)

In [284]:
len(types)

562